In [1]:
import requests
import boto3
import json
import os
import sys
import argparse
import importlib
import transformers
import torch
import pathlib
import awswrangler as wr
from IPython.display import display
from sagemaker.huggingface.processing import HuggingFaceProcessor
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.processing import FrameworkProcessor
from sagemaker.sklearn.estimator import SKLearn
from sagemaker.workflow.steps import ProcessingStep
from sagemaker.workflow.pipeline_context import PipelineSession
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.session import get_execution_role
from huggingface_hub import create_repo, upload_file, login
from tqdm.auto import tqdm
import fsspec


# Adding ../01_modules or ./01_modules to the system path so that we can load modules from 
# there as well
if '__file__' in globals():
    script_dir = pathlib.Path(__file__).parent.resolve()
else:
    script_dir = pathlib.Path().absolute()
modules_path_in_dev = os.path.abspath(os.path.join(script_dir, '..', '01_modules'))
modules_path_in_prod = os.path.abspath(os.path.join(script_dir, '01_modules'))
if os.path.exists(modules_path_in_dev):
    sys.path.append(modules_path_in_dev)
if os.path.exists(modules_path_in_prod):
    sys.path.append(modules_path_in_prod)


# # Jupyter only reads a local module the first time after 
# # kernel start. Re-running a cell with 
# # "from mymodulename import *" would not change
# # anything, even if the imported module has since changed.
# # As a workaround, we need to directly load the module, 
# # use importlib.reload to reload it and then import * 
import utils
_ = importlib.reload(utils)
import config
_ = importlib.reload(config) 


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
config.py loaded: v0.1
utils.py loaded: v0.2.12
utils.py loaded: v0.2.12
config.py loaded: v0.1


In [3]:
login()

In [9]:
fs = fsspec.filesystem('s3')  # s3 / gcs / abfs / adl / oci / ...
subset = 'test'
data_dir = f's3://sagemaker-research-methodology-extraction/01_data/03_core/unified_works_{subset}/'
pattern = '*'
data_files = fs.glob(data_dir + pattern)

utils.ensure_path('temp_files')
# create_repo(destination_dataset, repo_type='dataset')
for data_file in tqdm(fs.glob(data_dir + pattern)):
    with fs.open(data_file, 'rb') as fileobj:
        path_in_repo = f'data/{subset}/{data_file[len(data_dir):]}'
        print(path_in_repo, type(fileobj))
        upload_file(
            path_or_fileobj=fileobj,
            path_in_repo=path_in_repo,
            repo_id=destination_dataset,
            repo_type="dataset",
        )

  0%|          | 0/21 [00:00<?, ?it/s]

data/test/903_232908_00111_au2h8_011ae08a-6999-49ff-bfe0-faeb769ea2b3 <class 's3fs.core.S3File'>


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:12                                                                                   │
│                                                                                                  │
│    9 │   with fs.open(data_file, 'rb') as fileobj:                                               │
│   10 │   │   path_in_repo = f'data/{subset}/{data_file[len(data_dir):]}'                         │
│   11 │   │   print(path_in_repo, type(fileobj))                                                  │
│ ❱ 12 │   │   upload_file(                                                                        │
│   13 │   │   │   path_or_fileobj=fileobj,                                                        │
│   14 │   │   │   path_in_repo=path_in_repo,                                                      │
│   15 │   │   │   repo_id=destination_dataset,                                                    │
│                                                                                                  │
│ /home/sagemaker-user/.conda/envs/python_311/lib/python3.11/site-packages/huggingface_hub/utils/_ │
│ validators.py:114 in _inner_fn                                                                   │
│                                                                                                  │
│   111 │   │   if check_use_auth_token:                                                           │
│   112 │   │   │   kwargs = smoothly_deprecate_use_auth_token(fn_name=fn.__name__, has_token=ha   │
│   113 │   │                                                                                      │
│ ❱ 114 │   │   return fn(*args, **kwargs)                                                         │
│   115 │                                                                                          │
│   116 │   return _inner_fn  # type: ignore                                                       │
│   117                                                                                            │
│                                                                                                  │
│ /home/sagemaker-user/.conda/envs/python_311/lib/python3.11/site-packages/huggingface_hub/hf_api. │
│ py:1669 in _inner                                                                                │
│                                                                                                  │
│    1666 │   │   │   return self.run_as_future(fn, self, *args, **kwargs)                         │
│    1667 │   │                                                                                    │
│    1668 │   │   # Otherwise, call the function normally                                          │
│ ❱  1669 │   │   return fn(self, *args, **kwargs)                                                 │
│    1670 │                                                                                        │
│    1671 │   _inner.is_future_compatible = True  # type: ignore                                   │
│    1672 │   return _inner  # type: ignore                                                        │
│                                                                                                  │
│ /home/sagemaker-user/.conda/envs/python_311/lib/python3.11/site-packages/huggingface_hub/hf_api. │
│ py:4705 in upload_file                                                                           │
│                                                                                                  │
│    4702 │   │   commit_message = (                                                               │
│    4703 │   │   │   commit_message if commit_message is not None else f"Upload {path_in_repo} wi │
│    4704 │   │   )                                                                                │
│ ❱  4705 │   │   operation = CommitOperationAdd(            

In [ ]:
import smart_open

bucket = 'sagemaker-research-methodology-extraction'
subset = 'test'
data_dir = f's3://sagemaker-research-methodology-extraction/01_data/03_core/unified_works_{subset}/'
for key, content in s3.iter_bucket(bucket, prefix=data_dir, workers=8):
    

In [10]:
import boto3
import os
from pathlib import Path
import time

s3_client = boto3.client('s3')
bucket_name = 'sagemaker-research-methodology-extraction'
timelogger = utils.TimeLogger()
destination_dataset = 'SteveAKopias/SemanticScholarCSFullTextWithOpenAlexTopics'
for subset in ['train', 'test', 'validation']:
    timelogger.log(f'START subset: {subset}')
    s3_folder_prefix = f'01_data/03_core/unified_works_{subset}'
    local_download_path = 'temp_files'
    utils.ensure_path(local_download_path)

    paginator = s3_client.get_paginator('list_objects_v2')
    file_entry_pages = paginator.paginate(Bucket=bucket_name, Prefix=s3_folder_prefix)

    for file_entry_page in file_entry_pages:
        if 'Contents' in file_entry_page:
            for file_entry in file_entry_page['Contents']:
                s3_key = file_entry['Key']
                relative_path = Path(s3_key).relative_to(s3_folder_prefix)
                path_in_repo = f'data/{subset}/{relative_path}.parquet'
                print('path_in_repo', path_in_repo)
                local_file_path = Path(local_download_path) / subset / relative_path

                local_file_path.parent.mkdir(parents=True, exist_ok=True)
                # print('relative_path.resolve()', str(local_file_path.resolve()), type(local_file_path.resolve()))

                s3_client.download_file(bucket_name, s3_key, str(local_file_path))
                # print(f"{subset}: Downloaded {s3_key} to {local_file_path}")
                upload_result = upload_file(
                    path_or_fileobj=local_file_path,
                    path_in_repo=path_in_repo,
                    repo_id=destination_dataset,
                    repo_type="dataset",
                )
                # print(f"{subset}: Uploaded {s3_key} to {path_in_repo}")
                if '/home/sagemaker-user/research_methodology_extraction/src/04_transformation/temp_files/' in str(local_file_path.resolve()):
                    os.remove(local_file_path)
                    # print(f"{subset}: Deleted {s3_key} at {local_file_path}")
                time.sleep(10)
    timelogger.log(f'END subset: {subset}')

 :: :: TIMELOGGER STARTED :: | since_start: 0.00 seconds | since_last: 0.00 seconds :: 
 :: START subset: train | since_start: 0.00 seconds | since_last: 0.00 seconds :: 
ensure_path(temp_files)
path_in_repo data/train/20250904_090806_00079_b7ug3_0938f420-d6b3-4492-8ef2-cc281f91629b.parquet


Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  ...938f420-d6b3-4492-8ef2-cc281f91629b:  21%|##        | 33.5MB /  160MB            

path_in_repo data/train/20250904_090806_00079_b7ug3_0f3c1b7e-5d39-44a9-82b4-d68825f7eaff.parquet


...

path_in_repo data/validation/20250904_090906_00063_n3ezr_fe5fd846-dd05-484a-9fd7-c9d772e03a27.parquet


Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  ...e5fd846-dd05-484a-9fd7-c9d772e03a27: 100%|##########| 19.9MB / 19.9MB            

 :: END subset: validation | since_start: 13.0 minutes, 4.48 seconds | since_last: 4.0 minutes, 8.59 seconds :: 


In [11]:
from datasets import load_dataset, DatasetDict, Dataset
dataset = load_dataset(destination_dataset)
dataset

Resolving data files:   0%|          | 0/21 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/21 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/21 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Loading dataset shards:   0%|          | 0/19 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'topic_index', 'topic_display_name', 'subfield_index', 'subfield_display_name', 'title', 'abstract', 'fulltext', 'subset'],
        num_rows: 316885
    })
    validation: Dataset({
        features: ['id', 'topic_index', 'topic_display_name', 'subfield_index', 'subfield_display_name', 'title', 'abstract', 'fulltext', 'subset'],
        num_rows: 39753
    })
    test: Dataset({
        features: ['id', 'topic_index', 'topic_display_name', 'subfield_index', 'subfield_display_name', 'title', 'abstract', 'fulltext', 'subset'],
        num_rows: 39792
    })
})